# 12 Factor App
> 12 Factor App

- toc: true 
- comments: true
- author: Ankush Agarwal
- categories: [12 Factor App, Design]

### I. Codebase
    One codebase tracked in revision control, many deploys
    Multiple apps sharing the same code is a violation of twelve-factor. 
    The solution here is to factor shared code into libraries which can be included through 
        the dependency manager.
    A deploy is a running instance of the app. This is typically a production site, and one or 
        more staging sites.
        
    Observations:
        Done for all apps

### II. Dependencies
    Explicitly declare and isolate dependencies
    The full and explicit dependency specification is applied uniformly to both production and development.
    
    Questions:
    Check if any app uses any server dependency for execution
        How easy/difficult to migrate it to another server
        Can we just run build command to do local setup 
        Tivoli agent on server ?
        Data Source configuration ?
        
    Observations:
        Maven take care of dependency in Java
        Virtual env in Python            

### III. Config
    Store config in the environment
    Strict separation of config from code. 
    Config varies substantially across deploys, code does not.    
    The twelve-factor app stores config in environment variables (often shortened to env vars or env). 
    Env vars are easy to change between deploys without changing any code; unlike config files, 
        there is little chance of them being checked into the code repo accidentally; 
        and unlike custom config files, or other config mechanisms such as Java System Properties, 
        they are a language- and OS-agnostic standard.    
    Based on environment application itself point to correct setting
    
    Questions:
        How to manage environment properties, we generally don't track them in codebase 
        Should we keep all properties in codebase with suffix ?
        
    Observations:
        Generally done for environment related properties 
        

### IV. Backing services
    Treat backing services as attached resources
    A backing service is any service the app consumes over the network as part of its normal operation. 
        Examples include datastores (such as MySQL or CouchDB), messaging/queueing systems 
        (such as RabbitMQ or Beanstalkd),SMTP services for outbound email (such as Postfix), 
        and caching systems (such as Memcached).
    To the app, both are attached resources, accessed via a URL or other locator/credentials 
        stored in the config.
    A deploy of the twelve-factor app should be able to swap out a local MySQL database with 
        one managed by a third party (such as Amazon RDS) without any changes to the app’s code.
        
    Observations:
        Generally done for all database, queue

### V. Build, release, run
    Strictly separate build and run stages    
    The twelve-factor app uses strict separation between the build, release, and run stages. 
        For example, it is impossible to make changes to the code at runtime, 
        since there is no way to propagate those changes back to the build stage.
        
    Observations:
        Generally done for all apps

### VI. Processes
    Execute the app as one or more stateless processes
    Twelve-factor processes are stateless and share-nothing. 
    Any data that needs to persist must be stored in a stateful backing service, typically a database.
    The twelve-factor app never assumes that anything cached in memory or on disk 
        will be available on a future request or job        
    Some web systems rely on “sticky sessions” – that is, caching user session data in memory
        of the app’s process and expecting future requests from the same visitor to be routed 
        to the same process. 
    Sticky sessions are a violation of twelve-factor and should never be used or relied upon. 
    Session state data is a good candidate for a datastore that offers time-expiration, 
        such as Memcached or Redis.
        
    Observation:
           Generally done for all apps

### VII. Port binding
    Export services via port binding
    The twelve-factor app is completely self-contained and does not rely on runtime injection 
        of a webserver into the execution environment to create a web-facing service
    The web app exports HTTP as a service by binding to a port, and listening to requests
        coming in on that port.        
    Spring Boot, apart from many other benefits, provides us with a default embedded application server. 
    Hence, the JAR we generated earlier using Maven is fully capable of executing in any 
        environment just by having a compatible Java runtime
        
    Observations:
        Work fine for Sprint Boot application - don't need any runtime container

### VIII. Concurrency
    Scale out via the process model
    Application must also be able to span multiple processes running on multiple physical machines.
    The share-nothing, horizontally partitionable nature of twelve-factor app processes means that 
        adding more concurrency is a simple and reliable operation.
        
    Observation:
        Genrally done for all handler applications
    
    Question:
        How to achieve it in Batch mode

### IX. Disposability
    Maximize robustness with fast startup and graceful shutdown
    The twelve-factor app’s processes are disposable, meaning they can be started or stopped 
        at a moment’s notice. 
    This facilitates fast elastic scaling, rapid deployment of code or config changes, 
        and robustness of production deploys.
    The application should expose idempotent services. 
    This gives the flexibility to stop, move, or spin new services at any time without any other considerations.
    
    Observations:
        Generally done for all applications
       
    Question:
        If any handler is processing request from queue will it stay in queue or thrown away.
        

### X. Dev/prod parity
    Keep development, staging, and production as similar as possible
    
    The time gap: A developer may work on code that takes days, weeks, or even months to go into production.
    The personnel gap: Developers write code, ops engineers deploy it.
    The tools gap: Developers may be using a stack like Nginx, SQLite, and OS X, while the production deploy 
        uses Apache, MySQL, and Linux

    The twelve-factor app is designed for continuous deployment by keeping the gap between 
        development and production small. Looking at the three gaps described above:        
    
    Make the time gap small: a developer may write code and have it deployed hours or even just minutes later.
    Make the personnel gap small: developers who wrote code are closely involved in deploying it and watching 
        its behavior in production.
    Make the tools gap small: keep development and production as similar as possible.
    
    All deploys of the app (developer environments, staging, production) should be using the same type and 
        version of each of the backing services.

    Observation:
        Time gap can be improved

### XI. Logs
    Treat logs as event streams
    A twelve-factor app never concerns itself with routing or storage of its output stream. 
    It should not attempt to write to or manage logfiles. 
    Instead, each running process writes its event stream, unbuffered, to stdout. 
    During local development, the developer will view this stream in the foreground of their 
        terminal to observe the app’s behavior.
    The capture, storage, curation, and archival of such stream should be handled by the execution environment.
    To begin with, we can use SLF4J to handle logging abstractly within our application. 
    Moreover, we can use a tool like Fluentd to collect the stream of logs from applications and backing services.
    This we can feed into Elasticsearch for storage and indexing. 
    Finally, we can generate meaningful dashboards for visualization in Kibana.
    
    Observations:
        Generally application log data
        
    Question:
        What are the alternative to achieve this.

### XII. Admin processes
    Run admin/management tasks as one-off processes
    The twelve-factor methodology strongly suggests keeping such admin scripts together with the 
        application codebase. 
    In doing so, it should follow the same principles as we apply to the main application codebase. 
    It's also advisable to use a built-in REPL tool of the execution environment to run such scripts 
        on production servers.